## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-16-20-57-13 +0000,nyt,"Henrike Naumann, Artist Set to Represent Germa...",https://www.nytimes.com/2026/02/16/arts/design...
1,2026-02-16-20-55-26 +0000,nypost,Long Island Rep Laura Gillen faces uphill batt...,https://nypost.com/2026/02/16/us-news/long-isl...
2,2026-02-16-20-52-26 +0000,nypost,Dad of slain football teen claims Mamdani is f...,https://nypost.com/2026/02/16/us-news/dad-of-s...
3,2026-02-16-20-50-36 +0000,startribune,Trump administration formally denies Minnesota...,https://www.startribune.com/trump-administrati...
4,2026-02-16-20-47-15 +0000,nyt,Police Respond to Shooting at a Rhode Island H...,https://www.nytimes.com/2026/02/16/us/hockey-g...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2540/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,20
63,new,12
208,california,8
254,iran,8
474,up,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
221,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...,60
193,2026-02-16-05-39-01 +0000,bbc,Trump's new world order has become real and Eu...,https://www.bbc.com/news/articles/cddn002g6qzo...,47
62,2026-02-16-18-07-29 +0000,wapo,"Popular anger burns in Iran after crackdown, a...",https://www.washingtonpost.com/world/2026/02/1...,45
75,2026-02-16-17-26-59 +0000,nypost,Trump tears into Gavin Newsom as California go...,https://nypost.com/2026/02/16/us-news/trump-sa...,45
63,2026-02-16-18-03-14 +0000,nyt,Iran Holds Exercises in Strait of Hormuz After...,https://www.nytimes.com/2026/02/16/world/middl...,44


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
221,60,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...
136,39,2026-02-16-13-44-07 +0000,cbc,"Dana Eden, co-creator of Israeli TV series Teh...",https://www.cbc.ca/news/world/dana-eden-israel...
27,34,2026-02-16-19-45-37 +0000,nypost,Canada announces Janice Charette as new chief ...,https://nypost.com/2026/02/16/business/canada-...
84,34,2026-02-16-16-57-58 +0000,cbc,"Obama clarifies comments on aliens being real,...",https://www.cbc.ca/news/world/obama-aliens-rea...
74,32,2026-02-16-17-28-44 +0000,nypost,Kremlin slams claims it killed top Putin criti...,https://nypost.com/2026/02/16/world-news/kreml...
49,32,2026-02-16-18-34-39 +0000,nypost,Desperate California importing oil from Bahama...,https://nypost.com/2026/02/16/us-news/desperat...
96,27,2026-02-16-16-05-58 +0000,bbc,Russian 'pick-up artist' accused of secretly f...,https://www.bbc.com/news/articles/c9wn5p299eko...
233,27,2026-02-15-21-32-00 +0000,wsj,Anthropic’s artificial-intelligence tool Claud...,https://www.wsj.com/politics/national-security...
22,25,2026-02-16-20-03-44 +0000,nypost,French Laundry star chef Thomas Keller wants W...,https://nypost.com/2026/02/16/us-news/french-l...
31,25,2026-02-16-19-28-28 +0000,nypost,Shocking video captures moment fake ICE agent ...,https://nypost.com/2026/02/16/us-news/video-sh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
